In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
%matplotlib inline

In [ ]:
# Set plotting style
plt.style.use('seaborn-v0_8-paper')  

sns.set_context("paper")  
plt.rcParams['figure.figsize'] = (10, 6)  
plt.rcParams['savefig.dpi'] = 300  

In [ ]:
# Import our custom modules
from bandit_algorithms import LinUCB, SlidingWindowLinUCB,LinUCBDecay, LinTS, EpsilonGreedy, SlidingDoublyRobustSoftmax, RidgeSoftmax
from bandit_experiment import BanditExperiment

In [ ]:
# Load the dataset
def load_data(file_path='fx_trading_dataset.json'):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

# Let's examine the data first
data = load_data()
print(f"Dataset contains {len(data)} trade instances")
print(f"First data entry: {data[0]}")

In [ ]:

currency_pairs = set([d['context']['currency_pair'] for d in data])
dates = set([d['context']['date'] for d in data])
times_of_day = set([d['context']['time_of_day'] for d in data])

print(f"Unique currency pairs: {currency_pairs}")
print(f"Number of unique dates: {len(dates)}")
print(f"Unique times of day: {times_of_day}")

# How many arms/strategies are available?
n_arms = len(data[0]['rewards'])
print(f"Number of arms/strategies: {n_arms}")

In [ ]:
# Cell for running single algorithms for inspecting hyperparamterer sensitivity

experiment = BanditExperiment(data)

experiment.add_algorithm(LinUCBDecay, alpha=0.4, decay=0.99985)

print("Running experiment...")
results = experiment.run()



In [ ]:
# Set up and run the experiment
experiment = BanditExperiment(data)

# Add algorithms with different parameters
experiment.add_algorithm(LinUCB, alpha=0.25)
experiment.add_algorithm(SlidingWindowLinUCB, alpha=0.25, window_size=500)
experiment.add_algorithm(LinUCBDecay, alpha= 0.25, decay=0.99985)
experiment.add_algorithm(LinTS, v=0.025)
experiment.add_algorithm(EpsilonGreedy, epsilon=0.025)
experiment.add_algorithm(SlidingDoublyRobustSoftmax, tau=0.025, window_size=500)
experiment.add_algorithm(RidgeSoftmax, tau = 0.025)
# Run the experiment
print("Running experiment...")
results = experiment.run()

In [ ]:
# For selection logs:

selection_log = experiment.algorithms[5].selection_log_df
experiment.algorithms[5].selection_log_df.head()

In [ ]:
# For update logs:
update_log = experiment.algorithms[5].update_log_df
#offset the update log by 1 step so it starts with 0
update_log['step'] = update_log['step'] - 1
update_log.head()

In [ ]:
merged_log = pd.merge(selection_log, update_log, on='step', how='outer')
merged_log.head()


In [ ]:
# Visualizations

## 1. Cumulative Reward Analysis
experiment.plot_cumulative_rewards()
plt.title("Cumulative Reward Comparison")

In [ ]:
# First get the figure from the experiment
fig = experiment.plot_cumulative_rewards()

# Then modify the current axes
ax = plt.gca()  # Get current axes

# Set x-axis limits to show only last 3000 steps
total_steps = len(experiment.results[list(experiment.results.keys())[0]]['cumulative_rewards'])
ax.set_xlim(total_steps - 365, total_steps)

# Calculate y-axis limits based only on actual algorithm rewards for the last 3000 steps
y_min = min(min(result['cumulative_rewards'][-700:]) for result in experiment.results.values())
y_max = max(max(result['cumulative_rewards'][-700:]) for result in experiment.results.values())
y_range = y_max - y_min
padding = y_range * 0.1  # 10% padding

# Set y-axis limits
ax.set_ylim(y_min - padding, y_max + padding)

# Add more grid lines
ax.grid(True, alpha=0.3, linestyle='--')

# Update the plot
plt.tight_layout()
plt.show()

In [ ]:
## 2. Cumulative Regret Analysis
experiment.plot_cumulative_regrets()
plt.title("Cumulative Regret Comparison")

In [ ]:
## 3. Arm Selection Distribution
experiment.plot_arm_selection_frequencies()
plt.suptitle("Arm Selection Frequencies by Algorithm")

In [ ]:
## 4. Arm Selection Over Time
#experiment.plot_arm_selections_over_time()
#plt.suptitle("Arm Selections Over Time")


In [ ]:
## 1. Summary Statistics
summary_stats = experiment.print_summary_statistics()
print("Summary Statistics:")
summary_stats

In [ ]:
## 2. Statistical Significance Testing
test_results, pairwise_comparisons = experiment.statistical_tests()
print("ANOVA Results:")
print(test_results)
if pairwise_comparisons is not None:
    print("\nPairwise Comparisons (Tukey's HSD):")
    pairwise_comparisons

In [ ]:
def show_algorithm_summaries(experiment):
    
    # Create a dictionary to store stats
    stats = {}
    
    for algo_name, result in experiment.results.items():
        rewards = result['obtained_rewards']
        stats[algo_name] = {
            'Mean Reward': np.mean(rewards),
            'Median Reward': np.median(rewards),
            'Std Dev': np.std(rewards),
            'Min Reward': np.min(rewards),
            'Max Reward': np.max(rewards),
            '25th Percentile': np.percentile(rewards, 25),
            '75th Percentile': np.percentile(rewards, 75),
            'Final Cumulative Reward': result['cumulative_rewards'][-1],
            'Final Cumulative Regret': result['cumulative_regrets'][-1]
        }
    
    # Convert to DataFrame for nice display
    summary_df = pd.DataFrame(stats).T  # Transpose to have algorithms as rows
    
    # Add a regret per step metric
    summary_df['Regret per Step'] = summary_df['Final Cumulative Regret'] / len(experiment.data)
    
    return summary_df

In [ ]:
pairwise_results = show_algorithm_summaries(experiment)
display(pairwise_results)

In [ ]:
## 4. Non-stationarity Adaptation Analysis
adaptation_metrics = experiment.calculate_adaptation_metrics()
if adaptation_metrics is not None:
    print("Adaptation Metrics Around Regime Shifts:")
    adaptation_metrics

In [ ]:
## 5. Cold Start Analysis
cold_start_metrics = experiment.analyze_cold_start()
if cold_start_metrics is not None and not cold_start_metrics.empty:
    print("Cold Start Adaptation Metrics:")
    display(cold_start_metrics)

    # Save to CSV
    cold_start_metrics.to_csv("cold_start_metrics.csv", index=False)
    print("Saved cold start metrics to 'cold_start_metrics.csv'.")
else:
    print("No cold start metrics returned.")

In [ ]:

# Unique currency pairs in dataset
currency_pairs = sorted(list(set(d['context']['currency_pair'] for d in data)))

# Set color palette (one per algorithm)
palette = sns.color_palette("Set2", n_colors=len(experiment.results))

# Setup figure with better spacing
fig = plt.figure(figsize=(16, 4 * len(experiment.results)), constrained_layout=True)
gs = gridspec.GridSpec(len(experiment.results), 1, figure=fig)

for i, (algo_name, result) in enumerate(experiment.results.items()):
    ax = fig.add_subplot(gs[i])
    
    # Organize data for seaborn violinplot
    plot_data = []
    plot_labels = []
    
    for j, entry in enumerate(data):
        if j < len(result['obtained_rewards']):
            cp = entry['context']['currency_pair']
            reward = result['obtained_rewards'][j]
            plot_data.append(reward)
            plot_labels.append(cp)

    # Create DataFrame for seaborn
    df = pd.DataFrame({
        'Reward': plot_data,
        'Currency Pair': plot_labels
    })

    sns.violinplot(
        data=df,
        x='Currency Pair',
        y='Reward',
        ax=ax,
        color=palette[i],       # one color per algorithm
        inner='point',          # show mean/median points
        width=0.9               # make violins wider
    )
    
    ax.set_title(f"{algo_name} - Reward Distribution by Currency Pair", fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('')
    ax.set_ylabel('Reward')
    ax.tick_params(axis='x', rotation=30)  # rotate x labels for clarity

# Tighten layout before adding title
plt.tight_layout()
plt.subplots_adjust(top=0.92)  # reduce space between suptitle and plots
plt.suptitle("Reward Distributions per Algorithm by Currency Pair", fontsize=16)

plt.show()
print("Violin plots complete.")